<a href="https://colab.research.google.com/github/chaithanya21/Sentiment-Analysis-using-Pyspark-on-Multi-Social-Media-Data/blob/master/SENTIMENTAL_ANALYSIS_OF_CUSTOMER_PRODUCT_REVIEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

In [0]:
%matplotlib inline

Reading the text
The text is held in a compressed format. Luckily, we can still read it line by line. The first word gives the label, so we have to convert that into a number and then take the rest to be the comment.

In [0]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [16]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cosmos98","key":"660c9b8fb3581cf8c3d1fe87f3a874cc"}'}

In [0]:
!cp kaggle.json ~/.kaggle/

In [18]:
!kaggle datasets list

ref                                                      title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
rajeevw/ufcdata                                          UFC-Fight historical data from 1993 to 2019          3MB  2019-07-05 09:58:02           7889  
gustavomodelli/forest-fires-in-brazil                    Forest Fires in Brazil                              31KB  2019-08-24 16:09:16          13727  
chirin/africa-economic-banking-and-systemic-crisis-data  Africa Economic, Banking and Systemic Crisis Data   14KB  2019-07-21 02:00:17           4695  
shuyangli94/food-com-recipes-and-user-interactions       Food.com Recipes and Interactions                  267MB  2019-11-08 01:18:21           4100  
ruslankl/european-union-lgbt-survey-2012                 EU LGBT Survey                 

In [21]:
!kaggle datasets download -d bittlingmayer/amazonreviews -w -f train.ft.txt.bz2 


100% 441M/443M [00:11<00:00, 51.0MB/s]
100% 443M/443M [00:11<00:00, 41.4MB/s]


In [23]:
!kaggle datasets download -d bittlingmayer/amazonreviews -w -f test.ft.txt.bz2 


 82% 41.0M/50.2M [00:00<00:00, 28.2MB/s]
100% 50.2M/50.2M [00:01<00:00, 49.4MB/s]


In [24]:
!ls

adc.json	   kaggle.json	test.ft.txt.bz2.zip
amazonreviews.zip  sample_data	train.ft.txt.bz2.zip


In [26]:
!unzip train.ft.txt.bz2.zip

Archive:  train.ft.txt.bz2.zip
  inflating: train.ft.txt.bz2        


In [27]:
!unzip test.ft.txt.bz2.zip

Archive:  test.ft.txt.bz2.zip
  inflating: test.ft.txt.bz2         


In [28]:
!ls

adc.json	   kaggle.json	test.ft.txt.bz2      train.ft.txt.bz2
amazonreviews.zip  sample_data	test.ft.txt.bz2.zip  train.ft.txt.bz2.zip


In [0]:
train_labels, train_texts = get_labels_and_texts('train.ft.txt.bz2')

In [0]:
test_labels, test_texts = get_labels_and_texts('test.ft.txt.bz2')

Text Preprocessing

In [0]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts

In [0]:
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [0]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)

In [0]:
train_DF=pd.DataFrame(train_texts)


In [41]:
train_DF.head()



,0
0,don t miss it i don t know if i ve ever been...
1,durability a problem my kids have been playi...
2,excellent for what it is blood the last vamp...
3,poor quality the images on the paper are blur...
4,pretend and play school this is a wonderful p...


In [42]:
len(train_DF)

2880000